# Behavioral Cloning

Hereafter We present the information for Cloning driving behavior using Deep Learning. The code will be presented separately in the repository in the file `model.py` and different packages will be used to process training data gathered from Udacity's simulator. Images gathered will be extended to provide better traininig. Images were saved to an S3 bucket to provide access from multiple computers to craft the model separately from the training instance.

# Quality of code

## Is the code functional?

> The model provided can be used to successfully operate the simulation.

## Is the code usable and readable?

> The code in `model.py` uses a Python generator, if needed, to generate data for training rather than storing the training data in memory. The `model.py` code is clearly organized and comments are included where needed.

# Model Architecture and Training Strategy

## Has an appropriate model architecture been employed for the task?

> The neural network uses convolution layers with appropriate filter sizes. Layers exist to introduce nonlinearity into the model. The data is normalized in the model.

## Has an attempt been made to reduce overfitting of the model?

> Train/validation/test splits have been used, and the model uses dropout layers or other methods to reduce overfitting.

## Have the model parameters been tuned appropriately?

> Learning rate parameters are chosen with explanation, or an Adam optimizer is used.

## Is the training data chosen appropriately?

> Training data has been chosen to induce the desired behavior in the simulation (i.e. keeping the car on the track).

# Architecture and Training Documentation

## Is the solution design documented?

> The README thoroughly discusses the approach taken for deriving and designing a model architecture fit for solving the given problem.

## Is the model architecture documented?

> The README provides sufficient details of the characteristics and qualities of the architecture, such as the type of model used, the number of layers, the size of each layer. Visualizations emphasizing particular qualities of the architecture are encouraged.

## Is the creation of the training dataset and training process documented?

> The README describes how the model was trained and what the characteristics of the dataset are. Information such as how the dataset was generated and examples of images from the dataset should be included.

To assemble the data we use multiple use cases, we need to be able to extend the data easily so we create some scripts that rebuilds the labels.

In [2]:
import os

print(os.listdir('./data')) 

['track1', 'track2']


# Simulation

## Is the car able to navigate correctly on test data?

> No tire may leave the drivable portion of the track surface. The car may not pop up onto ledges or roll over any surfaces that would otherwise be considered unsafe (if humans were in the vehicle).